#                                                                                                           PowerSys

# States

In [ ]:
from typing import TypedDict, Optional
import pandas as pd
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from dateutil import parser as dateparser
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import base64
from langgraph.graph import StateGraph


# Set environment BEFORE importing or using LangChain/LangSmith
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "cee322-eda"
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
class EDAState(TypedDict):
    """State for EDA agent with support for preprocessing and feature engineering"""
    # Input
    filepath: str
    
    # Data
    data: Optional[pd.DataFrame]
    preprocessed_data: Optional[pd.DataFrame]
    engineered_data: Optional[pd.DataFrame]
    
    # Analysis results
    summary_stats: Optional[dict]
    missing_info: Optional[dict]
    data_profile: Optional[dict]
    missing_percentage: float
    num_variables: int
    target_column: Optional[str] 
    
    # Visualization results
    plot_paths: list[str]
    visualizations: Optional[dict]
    viz_images: Optional[dict]  
    # Time series analysis
    ts_suitable: bool
    ts_method: Optional[str]
    ts_analysis: Optional[dict]
    model_state: Optional[dict]  
    
    # Forecasting results
    forecast_results: Optional[dict]
    forecast_images: Optional[dict]  
    
    # Reports
    eda_report: Optional[dict]
    report: Optional[dict]
    
    # Metadata
    errors: list[str]
    stop_processing: bool


# Enhanced State with Memory Management & Reducers
Adding proper state reducers and message tracking for persistent memory

In [ ]:
# Custom reducers for state management
from typing import Annotated
from langgraph.graph import add_messages
from langchain_core.messages import AIMessage
from datetime import datetime

def append_unique(current: list, update: list) -> list:
    """Append new items without duplicates"""
    if current is None:
        current = []
    if update is None:
        return current
    return current + [item for item in update if item not in current]

def merge_dicts(current: dict, update: dict) -> dict:
    """Deep merge dictionaries"""
    if current is None:
        current = {}
    if update is None:
        return current
    result = current.copy()
    for key, value in update.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            result[key] = {**result[key], **value}
        else:
            result[key] = value
    return result

# Helper to standardize memory augmentation for each node

def memory_update(step_name: str, content: str, *, errors=None, warnings=None, reflections=None, needs_revision: bool = False):
    """Return a dict containing standardized memory fields for a node step."""
    return {
        "messages": [AIMessage(content=content)],
        "steps_completed": [step_name],
        "current_step": step_name,
        "last_updated": datetime.now().isoformat(),
        "errors": errors or [],
        "warnings": warnings or [],
        "reflections": reflections or [],
        "needs_revision": needs_revision
    }

In [ ]:
import os
import uuid
from datetime import datetime
from typing import Annotated, TypedDict, Literal

# Enhanced state with memory management
class EDAState(TypedDict):
    filepath: str
    data: any
    preprocessed_data: any
    engineered_data: any
    summary_stats: str
    missing_info: str
    data_profile: str
    visualizations: str
    viz_images: dict
    forecast_results: dict
    forecast_images: dict
    ts_suitable: bool
    ts_method: str
    ts_analysis: str
    model_state: dict
    errors: Annotated[list, append_unique]
    stop_processing: bool
    report: str
    target_column: str
    missing_percentage: float
    num_variables: int
    feature_engineering_applied: bool
    feature_engineering_reason: str
    messages: Annotated[list, add_messages]
    session_id: str
    session_start_time: str
    last_updated: str
    steps_completed: Annotated[list, append_unique]
    current_step: str
    iteration_count: int
    max_iterations: int
    warnings: Annotated[list, append_unique]
    reflections: Annotated[list, append_unique]
    critique_history: Annotated[list, append_unique]
    needs_revision: bool
    revision_count: int
    quality_score: float
    data_quality_score: float
    analysis_quality_score: float
    forecast_quality_score: float
    checkpoint_tags: Annotated[list, append_unique]
    guardrail_passed: bool
    data_context: dict  # CRITICAL: Domain context (resolution, capacity, description, goal)

## Checkpoint Configuration (Persistent Memory)

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.serde.jsonplus import JsonPlusSerializer
import sqlite3

# Create checkpoint directory
checkpoint_dir = "/Users/albert/Desktop/lab7/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Initialize SQLite checkpointer for persistent memory
checkpoint_path = os.path.join(checkpoint_dir, "powersys_memory.db")

# Create connection and initialize checkpointer with pickle fallback for DataFrames
conn = sqlite3.connect(checkpoint_path, check_same_thread=False)
serde = JsonPlusSerializer(pickle_fallback=True)
checkpointer = SqliteSaver(conn, serde=serde)


## Session Management Functions

In [ ]:
from langchain_core.messages import HumanMessage

def run_analysis_with_memory(filepath: str, resume_session_id: str = None):
    """
    Run analysis with persistent memory checkpoint.
    
    Args:
        filepath: Path to data file
        resume_session_id: Optional session ID to resume (if not provided, creates new session)
    
    Returns:
        tuple: (result, session_id)
    """
    # Generate or use provided session ID
    session_id = resume_session_id if resume_session_id else str(uuid.uuid4())
    thread = {"configurable": {"thread_id": session_id}}
    
    # Initialize state
    initial_state = {
        "filepath": filepath,
        "data": None,
        "preprocessed_data": None,
        "engineered_data": None,
        "summary_stats": None,
        "missing_info": None,
        "data_profile": None,
        "missing_percentage": 0.0,
        "num_variables": 0,
        "target_column": None,
        "visualizations": {},
        "viz_images": {},
        "forecast_results": None,
        "forecast_images": {},
        "model_state": None,
        "report": None,
        "errors": [],
        "stop_processing": False,
        "ts_suitable": False,
        "ts_method": None,
        "ts_analysis": None,
        "feature_engineering_applied": False,
        "feature_engineering_reason": None,
        # Memory fields
        "messages": [HumanMessage(content=f"Analyze power generation data from: {filepath}")],
        "session_id": session_id,
        "session_start_time": datetime.now().isoformat(),
        "last_updated": datetime.now().isoformat(),
        "steps_completed": [],
        "current_step": "initialized",
        "iteration_count": 0,
        "max_iterations": 50,
        "warnings": [],
        # Reflection fields
        "reflections": [],
        "critique_history": [],
        "needs_revision": False,
        "revision_count": 0,
        "quality_score": 0.0,
        "data_quality_score": 0.0,
        "analysis_quality_score": 0.0,
        "forecast_quality_score": 0.0
    }
    
    # Run agent with checkpointing
    result = ts_agent.invoke(initial_state, thread)
    
    print(f"\n{'='*60}")
    print(f"Session ID: {session_id}")
    print(f"Completed steps: {len(result.get('steps_completed', []))}")
    print(f"{'='*60}\n")
    
    return result, session_id

def view_session_history(session_id: str):
    """View all checkpoints for a session"""
    thread = {"configurable": {"thread_id": session_id}}
    history = list(checkpointer.list(thread))
    
    print(f"\n{'='*60}")
    print(f"Session: {session_id}")
    print(f"Total checkpoints: {len(history)}")
    print(f"{'='*60}\n")
    
    for i, checkpoint_tuple in enumerate(history, 1):
        checkpoint_metadata = checkpoint_tuple.metadata
        print(f"Checkpoint {i}:")
        print(f"  Step: {checkpoint_metadata.get('step', 'N/A')}")
        print(f"  Timestamp: {checkpoint_metadata.get('writes', {})}")
        print()
    
    return history


## Updated Node Functions (with Memory Tracking)

The following node functions have been enhanced to:
- Log execution to `messages` field (using AIMessage)
- Track progress in `steps_completed`
- Update `current_step` and `last_updated` timestamps
- Preserve message history through state propagation

In [ ]:
def run_reflective_analysis(filepath: str, resume_session_id: str = None, data_context: dict = None):
    """
    Run analysis with reflection loops for iterative improvement (v2 with checkpoint tags).
    """
    session_id = resume_session_id if resume_session_id else str(uuid.uuid4())
    thread = {"configurable": {"thread_id": session_id}}

    initial_state = {
        "filepath": filepath,
        "data": None,
        "preprocessed_data": None,
        "engineered_data": None,
        "summary_stats": None,
        "missing_info": None,
        "data_profile": None,
        "missing_percentage": 0.0,
        "num_variables": 0,
        "target_column": None,
        "visualizations": {},
        "viz_images": {},
        "forecast_results": None,
        "forecast_images": {},
        "model_state": None,
        "report": None,
        "errors": [],
        "stop_processing": False,
        "ts_suitable": False,
        "ts_method": None,
        "ts_analysis": None,
        "feature_engineering_applied": False,
        "feature_engineering_reason": None,
        "messages": [HumanMessage(content=f"Starting reflective analysis on: {filepath}")],
        "session_id": session_id,
        "session_start_time": datetime.now().isoformat(),
        "last_updated": datetime.now().isoformat(),
        "steps_completed": [],
        "current_step": "initialized",
        "iteration_count": 0,
        "max_iterations": 50,
        "warnings": [],
        "reflections": [],
        "critique_history": [],
        "needs_revision": False,
        "revision_count": 0,
        "quality_score": 0.0,
        "data_quality_score": 0.0,
        "analysis_quality_score": 0.0,
        "forecast_quality_score": 0.0,
        "checkpoint_tags": [],
        "data_context": data_context or {"description": "wind power output data", "resolution": "15 minutes", "goal": "Day-ahead forecasting", "capacity": "99 MW"}
    }

    # Prefer v2 agent if available
    agent = globals().get("reflective_agent_v2") or globals().get("reflective_agent")
    # Increase recursion limit and use hard exit condition in check_after_forecast_reflection
    result = agent.invoke(initial_state, config={"recursion_limit": 50, **thread})

    print(f"\n{'='*70}")
    print("REFLECTIVE ANALYSIS COMPLETE (v2)")
    print(f"Session ID: {session_id}")
    print(f"Revision count: {result.get('revision_count', 0)}")
    print(f"Quality scores: D={result.get('data_quality_score',0):.2f} A={result.get('analysis_quality_score',0):.2f} F={result.get('forecast_quality_score',0):.2f} Overall={result.get('quality_score',0):.2f}")
    print(f"Checkpoint tags: {result.get('checkpoint_tags', [])}")
    print(f"{'='*70}\n")
    return result, session_id

In [ ]:
from langchain_core.messages import AIMessage

# Example: Updated load_data node with memory tracking
def load_data_with_memory(state: EDAState) -> EDAState:
    """Load data with conversation tracking"""
    step_name = "load_data"
    
    try:
        filepath = state["filepath"]
        ext = os.path.splitext(filepath)[1].lower()
        
        # Load data based on file type
        if ext == ".csv":
            data = pd.read_csv(filepath)
        elif ext in [".xls", ".xlsx"]:
            data = pd.read_excel(filepath)
        elif ext == ".txt":
            data = pd.read_csv(filepath, delimiter="\t")
        else:
            raise ValueError(f"Unsupported file format: {ext}")
        
        # Log success to conversation
        message = AIMessage(content=f"Successfully loaded {len(data)} rows from {os.path.basename(filepath)}")
        
        return {
            "data": data,
            "messages": [message],
            "steps_completed": [step_name],
            "current_step": step_name,
            "last_updated": datetime.now().isoformat()
        }
        
    except Exception as e:
        # Log error to conversation
        error_msg = f"Load error: {str(e)}"
        message = AIMessage(content=error_msg)
        
        return {
            "data": None,
            "errors": [error_msg],
            "messages": [message],
            "steps_completed": [step_name],
            "current_step": step_name,
            "last_updated": datetime.now().isoformat(),
            "stop_processing": True
        }


## Reflection and Critique Nodes

These nodes implement the observe-reflect-plan-act cycle for iterative improvement.

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize LLM for reflection
reflection_llm = ChatOpenAI(model="gpt-5-mini", temperature=0.3)

def mark_checkpoint(state: EDAState, label: str) -> EDAState:
    """Explicitly mark a logical checkpoint with a label so we can trace revision boundaries."""
    tags = state.get("checkpoint_tags", [])
    tags.append({"tag": label, "revision_count": state.get("revision_count", 0)})
    return memory_update(f"checkpoint_{label}", f"Checkpoint saved at {label} (rev={state.get('revision_count',0)})") | {"checkpoint_tags": tags}

def _reflection_memory(step: str, score: float, needs_revision: bool, critique: str, state: EDAState, label: str):
    entry = f"{label} Reflection (Score: {score:.2f}) needs_revision={needs_revision}: {critique}"[:6000]
    return memory_update(step, f"{label} reflection: score={score:.2f} needs_revision={needs_revision}", reflections=[entry], needs_revision=needs_revision, errors=state.get("errors", [])) | {
        "critique_history": [{"step": step, "score": score, "critique": critique}],
        "revision_count": state.get("revision_count", 0) + (1 if needs_revision else 0)
    }

def reflect_on_data_quality(state: EDAState) -> EDAState:
    step = "reflect_data"
    # BUG FIX: Avoid "or" with DataFrame - check explicitly for None/emptiness
    data = state.get("preprocessed_data")
    if data is None or (hasattr(data, 'empty') and data.empty):
        data = state.get("data")
    if data is None or (hasattr(data, 'empty') and data.empty):
        return _reflection_memory(step, 0.0, True, "No data available", state, "Data Quality") | {"data_quality_score": 0.0}
    
    missing_pct = state.get("missing_percentage", 0.0)
    num_rows = len(data); num_cols = state.get("num_variables", 0)
    prompt = f"""Evaluate dataset quality.
Rows: {num_rows}
Vars: {num_cols}
Missing%: {missing_pct:.2f}
Provide:
QUALITY_SCORE: <0-1>
NEEDS_REVISION: <yes/no>
REASONING: <brief>
RECOMMENDATIONS: <brief>
"""
    try:
        response = reflection_llm.invoke(prompt)
        critique = response.content
        quality_score = 0.5; needs_revision = False
        for line in critique.split('\n'):
            if 'QUALITY_SCORE:' in line:
                try: quality_score = float(line.split(':')[1].strip())
                except: pass
            if 'NEEDS_REVISION:' in line:
                needs_revision = 'yes' in line.lower()
        return _reflection_memory(step, quality_score, needs_revision, critique, state, "Data Quality") | {"data_quality_score": quality_score}
    except Exception as e:
        return _reflection_memory(step, 0.5, False, f"LLM error: {e}", state, "Data Quality") | {"data_quality_score": 0.5}

def reflect_on_analysis_quality(state: EDAState) -> EDAState:
    step = "reflect_analysis"
    fe_applied = state.get("feature_engineering_applied", False)
    fe_reason = state.get("feature_engineering_reason", "")
    num_vars = state.get("num_variables", 0)
    prompt = f"""Evaluate analysis quality.
Vars: {num_vars}
FeatureEngineeringApplied: {fe_applied}
Reason: {fe_reason}
Provide:
QUALITY_SCORE: <0-1>
NEEDS_REVISION: <yes/no>
REASONING: <brief>
RECOMMENDATIONS: <brief>
"""
    try:
        response = reflection_llm.invoke(prompt)
        critique = response.content
        quality_score = 0.5; needs_revision = False
        for line in critique.split('\n'):
            if 'QUALITY_SCORE:' in line:
                try: quality_score = float(line.split(':')[1].strip())
                except: pass
            if 'NEEDS_REVISION:' in line:
                needs_revision = 'yes' in line.lower()
        return _reflection_memory(step, quality_score, needs_revision, critique, state, "Analysis Quality") | {"analysis_quality_score": quality_score}
    except Exception as e:
        return _reflection_memory(step, 0.5, False, f"LLM error: {e}", state, "Analysis Quality") | {"analysis_quality_score": 0.5}

def reflect_on_forecast_quality(state: EDAState) -> EDAState:
    step = "reflect_forecast"
    forecast_results = state.get("forecast_results")
    if not forecast_results:
        return _reflection_memory(step, 0.0, True, "No forecast results", state, "Forecast Quality") | {"forecast_quality_score": 0.0, "quality_score": 0.0}
    rf_metrics = forecast_results.get("methods", {}).get("random_forest", {}).get("metrics", {})
    dt_metrics = forecast_results.get("methods", {}).get("decision_tree", {}).get("metrics", {})
    rf_r2 = rf_metrics.get("R2", -999); dt_r2 = dt_metrics.get("R2", -999)
    best_r2 = max(rf_r2, dt_r2)
    prompt = f"""Evaluate forecast quality.
RF_R2: {rf_r2}
DT_R2: {dt_r2}
Best_R2: {best_r2}
Provide:
QUALITY_SCORE: <0-1>
NEEDS_REVISION: <yes/no>
REASONING: <brief>
RECOMMENDATIONS: <brief>
"""
    try:
        response = reflection_llm.invoke(prompt)
        critique = response.content
        quality_score = 0.5; needs_revision = False
        for line in critique.split('\n'):
            if 'QUALITY_SCORE:' in line:
                try: quality_score = float(line.split(':')[1].strip())
                except: pass
            if 'NEEDS_REVISION:' in line:
                needs_revision = 'yes' in line.lower()
        if best_r2 < 0:
            needs_revision = True
            quality_score = min(quality_score, 0.3)
        mem = _reflection_memory(step, quality_score, needs_revision, critique, state, "Forecast Quality")
        return mem | {"forecast_quality_score": quality_score, "quality_score": quality_score}
    except Exception as e:
        return _reflection_memory(step, 0.5, False, f"LLM error: {e}", state, "Forecast Quality") | {"forecast_quality_score": 0.5, "quality_score": 0.5}

print("Reflection nodes + checkpoint marker updated (DataFrame ambiguity bug fixed)")

## Guardrail Node: Physical Constraint Validation

In [ ]:
def guardrail_validation(state: EDAState) -> EDAState:
    """
    Validate physical constraints:
    1. Predictions must not exceed nominal capacity
    2. MAE, RMSE, R2 must be > 0
    3. All power output data must be >= 0 (renewable energy constraint)
    """
    step = "guardrail_validation"
    violations = []
    warnings = []
    
    # Extract capacity from data profile
    data_profile = state.get("data_profile", {})
    capacity = data_profile.get("nominal_capacity")
    
    # Check data non-negativity
    data = state.get("engineered_data")
    if data is None:
        data = state.get("preprocessed_data")
    if data is None:
        data = state.get("data")
    
    if data is not None:
        target_col = state.get("target_column")
        if target_col and target_col in data.columns:
            min_value = data[target_col].min()
            if min_value < 0:
                violations.append(f"Negative power output detected: min={min_value:.3f} (renewable energy must be >= 0)")
    
    # Check forecast results
    forecast_results = state.get("forecast_results")
    if forecast_results:
        methods = forecast_results.get("methods", {})
        
        for method_name, method_data in methods.items():
            metrics = method_data.get("metrics", {})
            predictions = method_data.get("predictions")
            
            # Validate metrics > 0
            mae = metrics.get("MAE", 0)
            rmse = metrics.get("RMSE", 0)
            r2 = metrics.get("R2", -999)
            
            if mae < 0:
                violations.append(f"{method_name}: MAE={mae:.3f} is negative (invalid)")
            if rmse < 0:
                violations.append(f"{method_name}: RMSE={rmse:.3f} is negative (invalid)")
            if r2 < 0:
                warnings.append(f"{method_name}: R2={r2:.3f} is negative (poor model fit)")
            
            # Validate predictions <= capacity
            if predictions is not None and capacity is not None:
                max_pred = np.max(predictions)
                if max_pred > capacity:
                    violations.append(f"{method_name}: max prediction={max_pred:.3f} exceeds capacity={capacity} MW")
            
            # Validate predictions >= 0
            if predictions is not None:
                min_pred = np.min(predictions)
                if min_pred < 0:
                    violations.append(f"{method_name}: min prediction={min_pred:.3f} is negative (renewable energy must be >= 0)")
    
    # Determine if revision needed
    needs_revision = len(violations) > 0
    
    # Build guardrail report
    if violations:
        violation_text = "; ".join(violations)
        content = f"Guardrail violations detected: {violation_text}"
    else:
        content = "All physical constraints satisfied"
    
    result = memory_update(
        step, 
        content,
        warnings=warnings,
        errors=violations if needs_revision else []
    )
    
    result["needs_revision"] = needs_revision
    result["guardrail_passed"] = not needs_revision
    
    if needs_revision:
        result["revision_count"] = state.get("revision_count", 0) + 1
    
    return result

## LLM Vision Reflection Node: Image Analysis and Content Verification

In [ ]:
def export_images_to_workspace(state: EDAState, output_dir: str = "/Users/albert/Desktop/lab7/outputs") -> dict:
    """
    Export all generated images (viz_images + forecast_images) to workspace files
    for LLM vision analysis to read from disk
    """
    import os
    
    step_name = "export_images"
    
    viz_images = state.get('viz_images', {})
    forecast_images = state.get('forecast_images', {})
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    exported_files = {}
    
    # Export EDA visualization images
    for fig_name, img_base64 in viz_images.items():
        try:
            img_bytes = base64.b64decode(img_base64)
            filename = f"{output_dir}/figure_1_{fig_name}.png"
            with open(filename, 'wb') as f:
                f.write(img_bytes)
            exported_files[fig_name] = filename
        except Exception as e:
            print(f"Error exporting {fig_name}: {e}")
    
    # Export forecast images
    for fig_name, img_base64 in forecast_images.items():
        try:
            img_bytes = base64.b64decode(img_base64)
            filename = f"{output_dir}/figure_3_{fig_name}.png"
            with open(filename, 'wb') as f:
                f.write(img_bytes)
            exported_files[fig_name] = filename
        except Exception as e:
            print(f"Error exporting {fig_name}: {e}")
    
    out = {
        "exported_images": exported_files,
        "export_directory": output_dir
    }
    msg = f"Images exported: {len(exported_files)} files saved to {output_dir}"
    out.update(memory_update(step_name, msg))
    
    return out


def analyze_visualizations_with_vision(state: EDAState) -> EDAState:
    """
    Use LLM vision capabilities to analyze generated images:
    - Detect and verify figure titles
    - Analyze chart content and data patterns
    - Identify anomalies and outliers
    - Extract metrics and performance indicators
    - Provide quality assessment
    """
    step_name = "vision_analysis"
    
    exported_images = state.get('exported_images', {})
    export_dir = state.get('export_directory', '')
    
    if not exported_images:
        return memory_update(
            step_name,
            "No exported images available for vision analysis",
            warnings=["No visualizations to analyze"]
        )
    
    vision_llm = ChatOpenAI(model="gpt-4-vision", temperature=0.2)
    
    analysis_results = {}
    
    for fig_name, file_path in exported_images.items():
        try:
            with open(file_path, 'rb') as img_file:
                img_bytes = img_file.read()
                img_base64 = base64.b64encode(img_bytes).decode()
            
            if 'forecast' in fig_name or 'figure_3' in file_path:
                analysis_prompt = """
Analyze this forecast visualization image in detail:

1. Title Verification: What is the exact title shown in the image? Should be "Figure 3: Forecast Performance Plot"
2. Chart Type: Identify the visualization type (time series plot with multiple lines)
3. Models Comparison: Compare the performance of different models shown
4. Metrics Extraction: Extract and list all visible metrics (MAE, RMSE, R² values)
5. Prediction Quality: Based on visual inspection, assess prediction accuracy
6. Anomaly Detection: Are there any significant prediction errors or anomalies?
7. Data Patterns: Describe the observed patterns in training vs test data
8. Legend Verification: Confirm all legend items are clearly visible and labeled
9. Quality Score: Rate visualization clarity and informativeness (1-10)
10. Recommendations: Suggest any improvements to the visualization or models

Provide structured analysis.
"""
            else:
                analysis_prompt = """
Analyze this exploratory data analysis (EDA) visualization image:

1. Title Verification: What is the exact title shown? Should be "Figure 1" or "Figure 2"
2. Visualization Type: What type of chart is this? (line plot, correlation matrix, etc.)
3. Data Content: Describe what data is being displayed
4. Key Patterns: What are the main patterns, trends, or relationships observed?
5. Data Quality: Are there any missing values, outliers, or data quality issues visible?
6. Statistical Observations: Any notable statistical properties (distributions, correlations, etc.)?
7. Axes Labels: Verify axis labels are clear and appropriate
8. Color Scheme: Is the color scheme effective for interpretation?
9. Quality Score: Rate the visualization (1-10) for clarity and informativeness
10. Insights: What insights can be drawn from this visualization?

Provide structured analysis.
"""
            
            with open(file_path, 'rb') as img_file:
                response = vision_llm.invoke([
                    HumanMessage(content=[
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{img_base64}"
                            }
                        },
                        {
                            "type": "text",
                            "text": analysis_prompt
                        }
                    ])
                ])
            
            analysis_results[fig_name] = {
                "status": "analyzed",
                "file_path": file_path,
                "analysis": response.content
            }
            
        except Exception as e:
            analysis_results[fig_name] = {
                "status": "error",
                "file_path": file_path,
                "error": str(e)
            }
    
    # Create summary report
    analysis_summary = []
    for fig_name, result in analysis_results.items():
        if result['status'] == 'analyzed':
            analysis_summary.append(f"### {fig_name.upper()}\n{result['analysis']}\n")
        else:
            analysis_summary.append(f"### {fig_name.upper()}\nError: {result.get('error', 'Unknown error')}\n")
    
    summary_text = "\n".join(analysis_summary)
    
    out = {
        "vision_analysis": analysis_results,
        "vision_analysis_summary": summary_text
    }
    
    msg = f"Vision analysis complete: {len(analysis_results)} images analyzed"
    out.update(memory_update(step_name, msg))
    
    return out

# Load data node
As data has many formats, so use try and except paradigm

In [ ]:
def load_data(state: EDAState) -> EDAState:
    step_name = "load_data"
    try:
        filepath = state["filepath"]
        ext = os.path.splitext(filepath)[1].lower()
        if ext == ".csv":
            data = pd.read_csv(filepath)
        elif ext in [".xls", ".xlsx"]:
            data = pd.read_excel(filepath)
        elif ext == ".txt":
            data = pd.read_csv(filepath, delimiter="\t")
        else:
            raise ValueError(f"Unsupported file format: {ext}")
        
        msg = AIMessage(content=f"Loaded {len(data)} rows, {len(data.columns)} columns from {os.path.basename(filepath)}")
        return {
            "data": data,
            "messages": [msg],
            "steps_completed": [step_name],
            "current_step": step_name,
            "last_updated": datetime.now().isoformat(),
            "errors": state.get("errors", [])
        }
    except Exception as e:
        msg = AIMessage(content=f"Failed to load data: {str(e)}")
        return {
            "data": None,
            "messages": [msg],
            "steps_completed": [step_name],
            "current_step": step_name,
            "last_updated": datetime.now().isoformat(),
            "errors": state.get("errors", []) + [f"Load error: {str(e)}"],
            "stop_processing": True
        }

# Analyze data

In [ ]:
# Analyze data
from langchain_core.messages import AIMessage

def analyze_data(state: EDAState) -> EDAState:
    step_name = "analyze"
    data = state.get("data")
    if data is None:
        return memory_update(step_name, "No data available for analysis", errors=state.get("errors", []) + ["Analyze: no data"], needs_revision=True)

    df = data.copy()
    # Detect time-like column
    time_col = None
    candidate_cols = []
    for col in df.columns:
        lc = col.lower()
        if any(k in lc for k in ("time", "date", "datetime", "ts")):
            candidate_cols.append(col)
    for col in df.select_dtypes(include=['object', 'string']).columns:
        if col not in candidate_cols:
            sample = df[col].dropna().astype(str).head(10).tolist()
            parsed = 0
            for v in sample:
                try:
                    _ = dateparser.parse(v)
                    parsed += 1
                except Exception:
                    break
            if parsed >= min(3, len(sample)) and len(sample) > 0:
                candidate_cols.append(col)
    if len(candidate_cols) > 0:
        candidate_cols.sort(key=lambda c: (0 if any(k in c.lower() for k in ("time","date","datetime","ts")) else 1))
        time_col = candidate_cols[0]

    parsed_time_col = None
    if time_col:
        try:
            parsed = pd.to_datetime(df[time_col], utc=True, errors='coerce')
        except Exception:
            parsed = pd.to_datetime(df[time_col].astype(str), utc=True, errors='coerce')
        if parsed.isna().sum() > 0.5 * len(parsed):
            parsed = df[time_col].apply(lambda v: _safe_date_parse(v))
            parsed = pd.to_datetime(parsed, utc=True, errors='coerce')
        parsed_time_col = parsed
        df['_parsed_time'] = parsed_time_col
        bad = df['_parsed_time'].isna().sum()
        if bad:
            pass
        df = df.sort_values('_parsed_time', ascending=True).reset_index(drop=True)

    numeric_cols = df.select_dtypes(include=['number']).columns

    summary_stats = {}
    for col in numeric_cols:
        summary_stats[col] = {
            "mean": float(df[col].mean()),
            "median": float(df[col].median()),
            "std": float(df[col].std()),
            "min": float(df[col].min()),
            "max": float(df[col].max()),
            "count": int(df[col].count())
        }

    data_profile = {
        "total_rows": len(df),
        "total_columns": len(df.columns),
        "column_details": []
    }

    for col in df.columns:
        col_info = {
            "name": col,
            "dtype": str(df[col].dtype),
            "missing_count": int(df[col].isnull().sum()),
            "missing_percent": float(round(df[col].isnull().sum() / len(df) * 100, 2)),
            "non_null_count": int(df[col].count())
        }
        if col in numeric_cols:
            col_info.update({
                "mean": float(df[col].mean()),
                "median": float(df[col].median()),
                "std": float(df[col].std()),
                "min": float(df[col].min()),
                "max": float(df[col].max()),
                "q25": float(df[col].quantile(0.25)),
                "q75": float(df[col].quantile(0.75))
            })
        else:
            col_info.update({
                "unique_count": int(df[col].nunique()),
                "top_value": str(df[col].mode()[0]) if len(df[col].mode()) > 0 else None
            })
        data_profile["column_details"].append(col_info)

    data_profile["missing_data_summary"] = {
        "columns_with_missing": sum(1 for col_info in data_profile["column_details"] if col_info["missing_count"] > 0),
        "total_missing_cells": int(df.isnull().sum().sum()),
        "total_cells": int(len(df) * len(df.columns)),
        "missing_percent": float(round(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100, 2))
    }

    timespan_days = None
    enough_history = True
    if parsed_time_col is not None:
        times = pd.to_datetime(df['_parsed_time'], utc=True)
        if times.dropna().shape[0] >= 2:
            timespan = times.max() - times.min()
            timespan_days = float(timespan / pd.Timedelta(days=1))
            enough_history = timespan_days >= 7.0
        else:
            timespan_days = 0.0
            enough_history = False
    else:
        enough_history = False

    data_profile['timeseries'] = {
        'time_column': time_col,
        'timespan_days': timespan_days,
        'enough_history': enough_history
    }

    target_col = None
    for col in numeric_cols:
        if 'power' in col.lower() or 'generation' in col.lower() or 'mw' in col.lower():
            target_col = col
            break
    if target_col is None and len(numeric_cols) > 0:
        target_col = numeric_cols[0]

    missing_pct = data_profile.get('missing_data_summary', {}).get('missing_percent', 0)
    num_vars = len(numeric_cols)
    out = {
        "summary_stats": summary_stats,
        "data_profile": data_profile,
        "data": df,
        "missing_percentage": missing_pct,
        "num_variables": num_vars,
        "target_column": target_col,
        "stop_processing": False,
        "errors": state.get("errors", [])
    }

    timespan_days_val = data_profile.get('timeseries', {}).get('timespan_days', 0)
    if timespan_days_val is not None and timespan_days_val < 7.0:
        out["errors"] = out["errors"] + [f"Insufficient time series data: only {timespan_days_val:.1f} days (need >= 7 days for forecasting)"]
        out["stop_processing"] = True

    msg_summary = f"Analysis complete: {len(df)} rows, {num_vars} numeric vars, missing {missing_pct:.2f}%, target={target_col}, timespan_days={timespan_days_val}" + (" (insufficient history)" if out.get("stop_processing") else "")
    out.update(memory_update(step_name, msg_summary, errors=out.get("errors", []), needs_revision=out.get("stop_processing", False)))
    return out

from dateutil import parser

def _safe_date_parse(v):
    try:
        return parser.parse(str(v))
    except Exception:
        return pd.NaT

# Visualize Data: part of EDA

In [ ]:
# Data Visualization Node
def visualize_data(state: EDAState) -> EDAState:
    step_name = "visualize"
    
    # CRITICAL: Early exit if stop_processing flag is set (e.g., insufficient data)
    if state.get("stop_processing", False):
        out = {
            "plot_paths": [],
            "visualizations": {},
            "viz_images": {},
            "stop_processing": True,
            "errors": state.get("errors", [])
        }
        out.update(memory_update(step_name, "Visualization skipped: stop_processing flag is set", errors=out["errors"]))
        return out
    
    data = state.get("data")
    if data is None:
        base = {
            "plot_paths": [],
            "visualizations": {},
            "viz_images": {},
            "missing_percentage": state.get("missing_percentage"),
            "num_variables": state.get("num_variables"),
            "stop_processing": state.get("stop_processing", False),
            "preprocessed_data": state.get("preprocessed_data"),
            "engineered_data": state.get("engineered_data")
        }
        base.update(memory_update(step_name, "No data available for visualization", warnings=["Visualization skipped (no data)"]))
        return base

    data_profile = state.get('data_profile', {})
    timeseries_info = data_profile.get('timeseries', {})
    time_column = timeseries_info.get('time_column')

    numeric_cols = [col for col in data.select_dtypes(include=['number']).columns if not col.startswith('_')]

    viz_descriptions = {}
    viz_images = {}

    if time_column and '_parsed_time' in data.columns:
        power_col = None
        for col in numeric_cols:
            if 'power' in col.lower() or 'generation' in col.lower() or 'mw' in col.lower():
                power_col = col
                break
        if power_col is None and len(numeric_cols) > 0:
            power_col = numeric_cols[0]
        if power_col:
            fig, ax = plt.subplots(figsize=(12, 5))
            ax.plot(data['_parsed_time'], data[power_col], marker='o', linewidth=2, markersize=4, color='#8ECFC9')
            ax.set_xlabel(time_column, fontsize=12)
            ax.set_ylabel(power_col, fontsize=12)
            fig_name = 'Power Vs Time'
            ax.set_title(f'Figure 1: {fig_name}', fontsize=14, fontweight='bold')
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            buf = io.BytesIO()
            fig.savefig(buf, format='png', dpi=100, bbox_inches='tight')
            buf.seek(0)
            img_base64 = base64.b64encode(buf.read()).decode()
            buf.close()
            plt.close(fig)
            viz_descriptions['power_vs_time'] = {
                'title': f'{power_col} over Time',
                'description': f'Time series visualization of {power_col}'
            }
            viz_images['power_vs_time'] = img_base64

    if len(numeric_cols) > 1:
        corr_data = data[numeric_cols].corr()
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(corr_data, annot=True, fmt='.3f', cmap='coolwarm', center=0, square=True, ax=ax, cbar_kws={'label': 'Correlation'}, vmin=-1, vmax=1)
        ax.set_title('Figure 2: Correlation Matrix', fontsize=14, fontweight='bold')
        plt.tight_layout()
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=100, bbox_inches='tight')
        buf.seek(0)
        img_base64 = base64.b64encode(buf.read()).decode()
        buf.close()
        plt.close(fig)
        viz_descriptions['correlation_matrix'] = {
            'title': 'Correlation Matrix',
            'description': 'Pairwise correlations between numeric features'
        }
        viz_images['correlation_matrix'] = img_base64

    out = {
        "plot_paths": [],
        "visualizations": viz_descriptions,
        "viz_images": viz_images,
        "missing_percentage": state.get("missing_percentage"),
        "num_variables": state.get("num_variables")
    }
    out.update(memory_update(step_name, f"Visualization complete: {len(viz_images)} plot(s) generated"))
    return out

# Data Preprocessing
- interpolation


In [ ]:
# Data Preprocessing Node
def data_preprocessing(state: EDAState) -> EDAState:
    step_name = "preprocess"
    
    # Early exit if stop_processing flag is set (e.g., insufficient data)
    if state.get("stop_processing", False):
        out = {
            "preprocessed_data": None,
            "stop_processing": True,
            "errors": state.get("errors", [])
        }
        out.update(memory_update(step_name, "Preprocessing skipped: stop_processing flag is set", errors=out["errors"]))
        return out
    
    data = state.get("data")
    data_profile = state.get("data_profile", {})
    if data is None:
        return memory_update(step_name, "No data for preprocessing", errors=state.get("errors", []) + ["Preprocess: no data"], needs_revision=True)

    missing_pct = data_profile.get('missing_data_summary', {}).get('missing_percent', 0)
    num_vars = len(data.select_dtypes(include=['number']).columns)

    if missing_pct > 10:
        out = {
            "preprocessed_data": None,
            "stop_processing": True,
            "missing_percentage": missing_pct,
            "num_variables": num_vars,
            "errors": state.get("errors", []) + [f"Data contains {missing_pct}% missing values (>10%). Processing stopped."]
        }
        out.update(memory_update(step_name, f"Preprocessing halted: missing {missing_pct:.2f}% exceeds threshold", errors=out["errors"], needs_revision=True))
        return out

    preprocessed = data.copy()
    if missing_pct > 0:
        numeric_cols = preprocessed.select_dtypes(include=['number']).columns
        for col in numeric_cols:
            if preprocessed[col].isnull().sum() > 0:
                preprocessed[col] = preprocessed[col].ffill()
                preprocessed[col] = preprocessed[col].bfill()

    out = {
        "preprocessed_data": preprocessed,
        "stop_processing": False,
        "missing_percentage": missing_pct,
        "num_variables": num_vars
    }
    msg = f"Preprocessing complete: missing {missing_pct:.2f}% handled" if missing_pct > 0 else "Preprocessing complete: no missing values"
    out.update(memory_update(step_name, msg))
    return out

# Feature Engineering
- create more features for single variable data

In [ ]:
# Feature Engineering Node with Relevance Check
import numpy as np

def feature_engineering(state: EDAState) -> EDAState:
    step_name = "feature_eng"
    data = state.get("preprocessed_data")
    data_profile = state.get("data_profile", {})

    if data is None or state.get("stop_processing"):
        out = {"engineered_data": data, "feature_engineering_applied": False}
        out.update(memory_update(step_name, "Feature engineering skipped: no preprocessed data"))
        return out

    time_col = data_profile.get('timeseries', {}).get('time_column')
    if time_col is None:
        out = {"engineered_data": data, "feature_engineering_applied": False}
        out.update(memory_update(step_name, "Feature engineering skipped: no time column"))
        return out

    df = data.copy()

    if '_parsed_time' not in df.columns:
        df['_parsed_time'] = pd.to_datetime(df[time_col], utc=True, errors='coerce')

    time_idx = df['_parsed_time']
    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    target_col = None
    for col in numeric_cols:
        if 'power' in col.lower() or 'generation' in col.lower() or 'mw' in col.lower():
            target_col = col
            break
    if target_col is None and len(numeric_cols) > 0:
        target_col = numeric_cols[0]
    if target_col is None:
        out = {"engineered_data": df, "feature_engineering_applied": False}
        out.update(memory_update(step_name, "Feature engineering skipped: no numeric target"))
        return out

    feature_cols = [col for col in numeric_cols if col != target_col and not col.startswith('_')]
    needs_engineering = False
    engineering_reason = ""

    if len(feature_cols) <= 2:
        needs_engineering = True
        engineering_reason = f"Sparse input data ({len(feature_cols)+1} numeric features). Performing comprehensive feature engineering."
    else:
        correlations = df[feature_cols + [target_col]].corr()[target_col]
        relevant_features = (correlations.abs() > 0.3).sum() - 1
        if relevant_features < max(2, len(feature_cols) // 3):
            needs_engineering = True
            engineering_reason = f"Low correlation with target ({relevant_features}/{len(feature_cols)} features |corr|>0.3). Performing feature engineering."
        else:
            needs_engineering = False
            engineering_reason = f"Sufficient relevant features ({relevant_features}/{len(feature_cols)} with |corr|>0.3). Skipping feature engineering."

    if not needs_engineering:
        out = {
            "engineered_data": df,
            "feature_engineering_applied": False,
            "feature_engineering_reason": engineering_reason
        }
        out.update(memory_update(step_name, f"FE skipped: {engineering_reason}"))
        return out

    df['hour'] = time_idx.dt.hour
    df['day'] = time_idx.dt.day
    df['weekday'] = time_idx.dt.dayofweek
    df['month'] = time_idx.dt.month
    df['year'] = time_idx.dt.year
    df['season'] = df['month'].apply(lambda x: (x-1)//3 + 1)

    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
    df['year_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['year_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    lags = [1, 2, 24, 168, 336]
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)

    windows = [3, 6, 12, 24, 168]
    for window in windows:
        df[f'{target_col}_rolling_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_rolling_std_{window}'] = df[target_col].rolling(window).std()
        df[f'{target_col}_rolling_min_{window}'] = df[target_col].rolling(window).min()
        df[f'{target_col}_rolling_max_{window}'] = df[target_col].rolling(window).max()

    df[f'{target_col}_roc'] = df[target_col].pct_change()
    df[f'{target_col}_diff_1'] = df[target_col].diff(1)
    df[f'{target_col}_diff_24'] = df[target_col].diff(24)
    df[f'{target_col}_cumsum_normalized'] = (df[target_col].cumsum() - df[target_col].cumsum().min()) / (df[target_col].cumsum().max() - df[target_col].cumsum().min() + 1e-8)

    for k in range(1, 4):
        df[f'{target_col}_fourier_daily_sin_{k}'] = np.sin(2 * np.pi * k * df['hour'] / 24)
        df[f'{target_col}_fourier_daily_cos_{k}'] = np.cos(2 * np.pi * k * df['hour'] / 24)
    for k in range(1, 3):
        df[f'{target_col}_fourier_weekly_sin_{k}'] = np.sin(2 * np.pi * k * df['weekday'] / 7)
        df[f'{target_col}_fourier_weekly_cos_{k}'] = np.cos(2 * np.pi * k * df['weekday'] / 7)

    df = df.dropna()

    out = {
        "engineered_data": df,
        "feature_engineering_applied": True,
        "feature_engineering_reason": engineering_reason
    }
    msg = f"Feature engineering applied: {engineering_reason}; final cols={df.shape[1]}"
    out.update(memory_update(step_name, msg))
    return out

# Validation

In [ ]:
# Comprehensive Validation Node
def validate_data(state: EDAState) -> EDAState:
    step_name = "validate"
    errors = state.get("errors", [])
    data = state.get("data")

    if data is None:
        out = {"errors": errors + ["CRITICAL: Data failed to load"], "stop_processing": True}
        out.update(memory_update(step_name, "Validation failed: no data", errors=out["errors"], needs_revision=True))
        return out

    if len(data) == 0:
        out = {"errors": errors + ["CRITICAL: Dataset is empty"], "stop_processing": True}
        out.update(memory_update(step_name, "Validation failed: empty dataset", errors=out["errors"], needs_revision=True))
        return out

    if len(data) < 50:
        errors.append(f"WARNING: Dataset has only {len(data)} rows. Minimum 50 rows recommended for robust analysis")

    missing_counts = data.isnull().sum()
    missing_pct = (missing_counts / len(data) * 100).round(2)
    missing_info = {
        col: {"count": int(missing_counts[col]), "percent": float(missing_pct[col])}
        for col in data.columns if missing_counts[col] > 0
    }

    total_missing_pct = (data.isnull().sum().sum() / (len(data) * len(data.columns)) * 100)
    if total_missing_pct == 100:
        out = {"errors": errors + ["CRITICAL: All data is missing"], "stop_processing": True}
        out.update(memory_update(step_name, "Validation failed: all data missing", errors=out["errors"], needs_revision=True))
        return out

    numeric_cols = data.select_dtypes(include=['number']).columns
    if len(numeric_cols) == 0:
        out = {"errors": errors + ["CRITICAL: No numeric columns found for analysis"], "stop_processing": True}
        out.update(memory_update(step_name, "Validation failed: no numeric columns", errors=out["errors"], needs_revision=True))
        return out

    if len(data.columns) != len(set(data.columns)):
        errors.append("WARNING: Duplicate column names detected")

    out = {"missing_info": missing_info, "errors": errors, "stop_processing": False}
    msg = f"Validation passed: rows={len(data)}, numeric_cols={len(numeric_cols)}, missing_cols={len(missing_info)}"
    out.update(memory_update(step_name, msg, errors=errors))
    return out

# Time Series Analysis & Forecasting Nodes

## Timeseries checker

In [ ]:
# Time Series Suitability Analysis Node

def analyze_timeseries_suitability(state: EDAState) -> EDAState:
    step_name = "ts_suitable"
    data_profile = state.get('data_profile', {})
    engineered_data = state.get('engineered_data')

    timeseries_info = data_profile.get('timeseries', {})
    time_column = timeseries_info.get('time_column')
    timespan_days = timeseries_info.get('timespan_days', 0)
    enough_history = timeseries_info.get('enough_history', False)

    ts_analysis = {
        'has_time_column': time_column is not None,
        'timespan_days': timespan_days,
        'enough_history': enough_history,
        'suitable_for_neural_network': False,
        'suitable_for_statistical': False,
        'recommended_method': None,
        'prerequisites_met': True,
        'method_rationale': ''
    }

    if not time_column:
        ts_analysis['prerequisites_met'] = False
        ts_analysis['method_rationale'] = 'No time column detected. Cannot perform time series forecasting. Consider regression models instead.'
        ts_analysis['recommended_method'] = 'regression'
        out = {
            'ts_suitable': False,
            'ts_method': 'regression',
            'ts_analysis': ts_analysis
        }
        out.update(memory_update(step_name, 'TS suitability: no time column -> regression', warnings=['No time column']))
        return out

    if timespan_days >= 30 and enough_history:
        ts_analysis['suitable_for_neural_network'] = True
        ts_analysis['suitable_for_statistical'] = True
        ts_analysis['recommended_method'] = 'neural_network'
        ts_analysis['method_rationale'] = f'Sufficient historical data ({timespan_days:.1f} days). Neural networks recommended.'
    elif timespan_days >= 7:
        ts_analysis['suitable_for_neural_network'] = False
        ts_analysis['suitable_for_statistical'] = True
        ts_analysis['recommended_method'] = 'statistical'
        ts_analysis['method_rationale'] = f'Limited data ({timespan_days:.1f} days). Statistical methods recommended.'
    else:
        ts_analysis['prerequisites_met'] = False
        ts_analysis['recommended_method'] = 'baseline'
        ts_analysis['method_rationale'] = f'Insufficient historical data ({timespan_days:.1f} days). Only baseline methods feasible.'

    out = {
        'ts_suitable': ts_analysis['suitable_for_neural_network'] or ts_analysis['suitable_for_statistical'],
        'ts_method': ts_analysis['recommended_method'],
        'ts_analysis': ts_analysis
    }
    msg = f"TS suitability: method={ts_analysis['recommended_method']} span={timespan_days:.1f}d suitable={out['ts_suitable']}"
    out.update(memory_update(step_name, msg, warnings=[] if out['ts_suitable'] else ["Limited/insufficient timespan"]))
    return out

# LLM Validate
- stop processing when the prerequisites are not met, and output report directly

In [ ]:
# LLM-based Data Sufficiency Validation Node

def llm_validate_data_sufficiency(state: EDAState) -> dict:
    step_name = "llm_validate"
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import HumanMessage

    data_profile = state.get('data_profile', {})
    timeseries_info = data_profile.get('timeseries', {})
    timespan_days = timeseries_info.get('timespan_days', 0)
    time_column = timeseries_info.get('time_column')
    total_rows = data_profile.get('total_rows', 0)

    # CRITICAL: Direct check for data sufficiency (not LLM-based)
    # If less than 7 days, immediately set stop_processing and skip all further analysis
    if timespan_days < 7:
        errors = state.get('errors', []) + [f"Insufficient time series data: only {timespan_days:.1f} days (need >= 7 days for forecasting)"]
        out = {"stop_processing": True, "errors": errors, "llm_validation": "insufficient"}
        out.update(memory_update(step_name, f"Data insufficient ({timespan_days:.1f} days < 7 days required)", errors=errors, needs_revision=True))
        return out

    # If sufficient data, optionally use LLM for validation commentary
    validation_context = f"""
    Dataset Information:
    - Total rows: {total_rows}
    - Time column: {time_column}
    - Timespan: {timespan_days:.2f} days
    - Minimum required for forecasting: 7 days

    Determine if this dataset has sufficient history for time series forecasting.

    Respond concisely with a description of the situation and a brief suggestion

    GUARDRAIL:
    - do not makeup data, avoid hallucination
    """
    try:
        llm = ChatOpenAI(model="gpt-5-nano", temperature=0)
        response = llm.invoke([HumanMessage(content=validation_context)])
        validation_result = response.content.strip()
    except Exception:
        validation_result = "sufficient"

    out = {"stop_processing": False, "llm_validation": validation_result}
    out.update(memory_update(step_name, f"LLM validate: sufficient ({timespan_days:.1f} days)"))
    return out

# Time Series Forecasting: random forest and decision trees
start with some simple methods


In [ ]:
# Time Series: Train Forecast Models
def train_forecast_model(state: EDAState) -> dict:
    step_name = "ts_train"
    engineered_data = state.get('engineered_data')
    target_col = state.get('target_column', 'Power MW')
    if engineered_data is None or engineered_data.empty:
        out = {'forecast_results': None, 'model_state': None, 'target_column': target_col}
        out.update(memory_update(step_name, "Training skipped: no engineered data", warnings=["No data for training"], needs_revision=True))
        return out

    feature_cols = [col for col in engineered_data.columns if col != target_col and not col.startswith('_') and engineered_data[col].dtype in ['float64', 'float32', 'int64', 'int32']]
    X = engineered_data[feature_cols].values
    y = engineered_data[target_col].values

    split_point = len(X)
    test_size = 0
    if '_parsed_time' in engineered_data.columns:
        timestamps = engineered_data['_parsed_time'].values
        last_timestamp = pd.Timestamp(timestamps[-1])
        cutoff_timestamp = last_timestamp - pd.Timedelta(days=1)
        for i in range(len(timestamps)):
            if pd.Timestamp(timestamps[i]) >= cutoff_timestamp:
                split_point = i
                break
        test_size = len(X) - split_point
    else:
        test_size = max(1, int(len(X) * 0.05))
        split_point = len(X) - test_size

    X_train, X_test = X[:split_point], X[split_point:]
    y_train, y_test = y[:split_point], y[split_point:]
    X_train = X_train.astype(float); X_test = X_test.astype(float)
    X_train[np.isinf(X_train)] = np.nan; X_test[np.isinf(X_test)] = np.nan
    train_mean = np.nanmean(X_train, axis=0)
    for i in range(X_train.shape[1]):
        nan_mask = np.isnan(X_train[:, i]); X_train[nan_mask, i] = train_mean[i]
        nan_mask_test = np.isnan(X_test[:, i]); X_test[nan_mask_test, i] = train_mean[i]

    rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    rf_mae = mean_absolute_error(y_test, rf_pred)
    rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
    rf_r2 = r2_score(y_test, rf_pred)

    dt_model = DecisionTreeRegressor(max_depth=15, random_state=42)
    dt_model.fit(X_train, y_train)
    dt_pred = dt_model.predict(X_test)
    dt_mae = mean_absolute_error(y_test, dt_pred)
    dt_rmse = np.sqrt(mean_squared_error(y_test, dt_pred))
    dt_r2 = r2_score(y_test, dt_pred)

    forecast_results = {
        'target_column': target_col,
        'test_size': test_size,
        'split_point': split_point,
        'methods': {
            'random_forest': {
                'method': 'Random Forest Regressor',
                'description': 'Ensemble method capturing non-linear relationships and feature interactions',
                'model': 'random_forest',
                'metrics': {'MAE': float(rf_mae), 'RMSE': float(rf_rmse), 'R2': float(rf_r2), 'test_size': len(y_test)},
                'parameters': {'n_estimators': 100, 'max_depth': 15, 'random_state': 42}
            },
            'decision_tree': {
                'method': 'Decision Tree Regressor',
                'description': 'Single decision tree capturing hierarchical feature relationships',
                'model': 'decision_tree',
                'metrics': {'MAE': float(dt_mae), 'RMSE': float(dt_rmse), 'R2': float(dt_r2), 'test_size': len(y_test)},
                'parameters': {'max_depth': 15, 'random_state': 42}
            }
        }
    }

    model_state = {
        'y_test': y_test,
        'y_train': y_train,
        'rf_pred': rf_pred,
        'dt_pred': dt_pred,
        'split_point': split_point,
        'test_size': test_size,
        'rf_mae': rf_mae,
        'rf_rmse': rf_rmse,
        'rf_r2': rf_r2,
        'dt_mae': dt_mae,
        'dt_rmse': dt_rmse,
        'dt_r2': dt_r2
    }

    out = {
        'forecast_results': forecast_results,
        'model_state': model_state,
        'target_column': target_col,
        'ts_analysis': state.get('ts_analysis'),
        'ts_method': state.get('ts_method'),
        'ts_suitable': state.get('ts_suitable'),
        'data_profile': state.get('data_profile'),
        'summary_stats': state.get('summary_stats'),
        'engineered_data': engineered_data,
        'visualizations': state.get('visualizations'),
        'viz_images': state.get('viz_images'),
        'forecast_images': {}
    }
    msg = f"Training complete: RF R2={rf_r2:.3f} MAE={rf_mae:.3f}; DT R2={dt_r2:.3f} MAE={dt_mae:.3f}" + (" (poor performance)" if rf_r2 < 0 or dt_r2 < 0 else "")
    out.update(memory_update(step_name, msg, warnings=["Low R2"] if (rf_r2 < 0 or dt_r2 < 0) else []))
    return out

# Visualize forecasting results

In [ ]:
# Time Series: Visualize Forecast Results

def visualize_forecast_results(state: EDAState) -> dict:
    step_name = "ts_visualize"
    model_state = state.get('model_state')
    forecast_results = state.get('forecast_results', {})
    target_col = state.get('target_column')
    forecast_images = {}

    if model_state is None:
        out = {
            'forecast_images': {},
            'forecast_results': forecast_results,
            'target_column': target_col,
            'ts_analysis': state.get('ts_analysis'),
            'ts_method': state.get('ts_method'),
            'ts_suitable': state.get('ts_suitable'),
            'data_profile': state.get('data_profile'),
            'summary_stats': state.get('summary_stats'),
            'engineered_data': state.get('engineered_data'),
            'visualizations': state.get('visualizations'),
            'viz_images': state.get('viz_images')
        }
        out.update(memory_update(step_name, "Forecast visualization skipped: no model state", warnings=["No model state"]))
        return out

    try:
        y_test = model_state['y_test']; rf_pred = model_state['rf_pred']; dt_pred = model_state['dt_pred']; y_train = model_state['y_train']
        split_point = model_state['split_point']; test_size = model_state['test_size']
        rf_mae = model_state['rf_mae']; rf_rmse = model_state['rf_rmse']; rf_r2 = model_state['rf_r2']
        dt_mae = model_state['dt_mae']; dt_rmse = model_state['dt_rmse']; dt_r2 = model_state['dt_r2']

        fig, ax = plt.subplots(figsize=(16, 6))
        engineered_data = state.get('engineered_data')
        data_profile = state.get('data_profile', {})
        timeseries_info = data_profile.get('timeseries', {})
        time_column = timeseries_info.get('time_column')

        all_timestamps = None
        if engineered_data is not None and '_parsed_time' in engineered_data.columns and time_column:
            all_timestamps = engineered_data['_parsed_time'].values

        display_points = 200
        total_available = len(y_train) + len(y_test)
        if total_available <= display_points:
            train_display_start = 0; train_display_end = len(y_train); test_display_all = True
        else:
            points_to_skip = total_available - display_points
            if points_to_skip >= len(y_train):
                train_display_start = len(y_train); train_display_end = len(y_train); test_display_all = True
            elif points_to_skip > 0:
                train_display_start = len(y_train) - (display_points - len(y_test)); train_display_end = len(y_train); test_display_all = True
            else:
                train_display_start = 0; train_display_end = len(y_train); test_display_all = True

        train_y_display = y_train[train_display_start:train_display_end]
        train_timestamps_display = None; test_timestamps_display = None
        if all_timestamps is not None:
            train_timestamps_display = all_timestamps[train_display_start:train_display_end]
            test_timestamps_display = all_timestamps[split_point:split_point + test_size]

        if train_timestamps_display is not None and test_timestamps_display is not None:
            if len(train_y_display) > 0:
                ax.plot(train_timestamps_display, train_y_display, 'o-', linewidth=2, markersize=4, label='Training Data', color='#95B8D1', alpha=0.6, zorder=2)
            ax.plot(test_timestamps_display, y_test, 'o-', linewidth=2.5, markersize=6, label='Actual Test Values', color='#2E86AB', alpha=0.9, zorder=4)
            ax.plot(test_timestamps_display, rf_pred, 's--', linewidth=2.5, markersize=5, label=f'RF MAE:{rf_mae:.3f} RMSE:{rf_rmse:.3f} R²:{rf_r2:.3f}', color='#00B894', alpha=0.85, zorder=3)
            ax.plot(test_timestamps_display, dt_pred, '^--', linewidth=2.5, markersize=5, label=f'DT MAE:{dt_mae:.3f} RMSE:{dt_rmse:.3f} R²:{dt_r2:.3f}', color='#FF6B6B', alpha=0.85, zorder=3)
            if len(test_timestamps_display) > 0:
                split_time = pd.Timestamp(test_timestamps_display[0])
                ax.axvline(x=split_time, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Train/Test Split', zorder=1)
            ax.set_xlabel('Time', fontsize=13, fontweight='bold'); plt.xticks(rotation=45, ha='right')
        else:
            if len(train_y_display) > 0:
                train_x_indices = np.arange(train_display_start, train_display_end)
                ax.plot(train_x_indices, train_y_display, 'o-', linewidth=2, markersize=4, label='Training Data', color='#95B8D1', alpha=0.6, zorder=2)
            test_x_indices = np.arange(train_display_end, train_display_end + len(y_test))
            ax.plot(test_x_indices, y_test, 'o-', linewidth=2.5, markersize=6, label='Actual Test Values', color='#2E86AB', alpha=0.9, zorder=4)
            ax.plot(test_x_indices, rf_pred, 's--', linewidth=2.5, markersize=5, label=f'RF MAE:{rf_mae:.3f} RMSE:{rf_rmse:.3f} R²:{rf_r2:.3f}', color='#00B894', alpha=0.85, zorder=3)
            ax.plot(test_x_indices, dt_pred, '^--', linewidth=2.5, markersize=5, label=f'DT MAE:{dt_mae:.3f} RMSE:{dt_rmse:.3f} R²:{dt_r2:.3f}', color='#FF6B6B', alpha=0.85, zorder=3)
            ax.axvline(x=train_display_end, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Train/Test Split', zorder=1)
            ax.set_xlabel('Sample Index', fontsize=13, fontweight='bold')

        ax.set_ylabel('Power Generation (MW)', fontsize=13, fontweight='bold')
        ax.set_title('Figure 3: Forecast Performance Plot', fontsize=15, fontweight='bold', pad=20)
        ax.legend(loc='best', fontsize=9, framealpha=0.95, ncol=2)
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.8)
        ax.set_facecolor('#F8F9FA'); fig.patch.set_facecolor('white')
        plt.tight_layout()
        buf = io.BytesIO(); fig.savefig(buf, format='png', dpi=100, bbox_inches='tight'); buf.seek(0)
        forecast_img_base64 = base64.b64encode(buf.read()).decode(); buf.close(); plt.close(fig)
        forecast_images['test_performance'] = forecast_img_base64
    except Exception as e:
        return_dict = {
            'forecast_images': forecast_images,
            'forecast_results': forecast_results,
            'target_column': target_col,
            'ts_analysis': state.get('ts_analysis'),
            'ts_method': state.get('ts_method'),
            'ts_suitable': state.get('ts_suitable'),
            'data_profile': state.get('data_profile'),
            'summary_stats': state.get('summary_stats'),
            'engineered_data': state.get('engineered_data'),
            'visualizations': state.get('visualizations'),
            'viz_images': state.get('viz_images')
        }
        return_dict.update(memory_update(step_name, f"Forecast visualization failed: {str(e)}", errors=state.get("errors", []) + [str(e)], needs_revision=True))
        return return_dict

    out = {
        'forecast_images': forecast_images,
        'forecast_results': forecast_results,
        'target_column': target_col,
        'ts_analysis': state.get('ts_analysis'),
        'ts_method': state.get('ts_method'),
        'ts_suitable': state.get('ts_suitable'),
        'data_profile': state.get('data_profile'),
        'summary_stats': state.get('summary_stats'),
        'engineered_data': state.get('engineered_data'),
        'visualizations': state.get('visualizations'),
        'viz_images': state.get('viz_images')
    }
    out.update(memory_update(step_name, f"Forecast visualization complete: images={len(forecast_images)}"))
    return out

# Final Report Generation with Error Handling

In [ ]:
def generate_final_report(state: EDAState) -> dict:
    step_name = "final_report"
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import HumanMessage

    # Extract data context for domain-aware report generation
    data_context = state.get('data_context', {})
    if not data_context:
        # Fallback: try to get from initial state defaults
        data_context = {"description": "time series data", "resolution": "unknown", "goal": "forecasting", "capacity": "unknown"}
    
    resolution = data_context.get('resolution', 'unknown')
    capacity = data_context.get('capacity', 'unknown')
    description = data_context.get('description', 'time series data')
    goal = data_context.get('goal', 'forecasting')
    
    data_context_str = f"This is {description} with {resolution} temporal resolution. Forecasting objective: {goal}. System nominal capacity: {capacity}."

    errors = state.get('errors', [])
    ts_analysis = state.get('ts_analysis') or {}
    forecast_results = state.get('forecast_results') or {}
    forecast_images = state.get('forecast_images') or {}
    data_profile = state.get('data_profile') or {}
    summary_stats = state.get('summary_stats') or {}
    engineered_data = state.get('engineered_data')
    visualizations = state.get('visualizations') or {}
    viz_images = state.get('viz_images') or {}

    timeseries_info = data_profile.get('timeseries', {})
    timespan_days = timeseries_info.get('timespan_days', 0)
    total_rows = data_profile.get('total_rows', 0)
    feature_count = engineered_data.shape[1] if engineered_data is not None else 0
    recommended_method = ts_analysis.get('recommended_method', 'statistical')

    feature_engineering_applied = state.get('feature_engineering_applied', False)
    feature_engineering_reason = state.get('feature_engineering_reason', '')
    original_feature_count = data_profile.get('total_columns', 0)

    missing_pct = state.get('missing_percentage', 0)
    missing_info = f"Missing values: {missing_pct:.2f}% of dataset. Interpolation method: forward-fill then backward-fill for numeric columns."

    # CRITICAL: If stop_processing is True, generate ONLY insufficient data report
    # NO visualizations, NO forecast results, NO images should be included
    if state.get('stop_processing', False) and len(errors) > 0:
        try:
            insufficient_prompt = f"""You are a data analyst. The dataset contains only {timespan_days:.2f} days of data ({total_rows} rows), which is insufficient for reliable time series forecasting (minimum 7 days required).

{missing_info}

Provide a brief 1-2 paragraph assessment of:
1. Why this dataset is unsuitable for forecasting
2. What additional data collection would help
3. Alternative analyses that could be performed on this limited data

Data summary: {total_rows} rows, {timespan_days:.2f} days timespan"""
            llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
            response = llm.invoke([HumanMessage(content=insufficient_prompt)])
            summary_text = response.content
        except Exception:
            summary_text = f"Dataset insufficient: {total_rows} rows over {timespan_days:.2f} days. Forecasting not recommended."
        
        # Return ONLY the insufficient data report with NO images
        report_obj = {
            'status': 'insufficient_data',
            'errors': errors,
            'forecast_summary': summary_text,
            'data_profile': data_profile,
            'ts_analysis': ts_analysis,
            'visualizations': {},  # Empty - no visualizations for insufficient data
            'viz_images': {},  # Empty - no images for insufficient data
            'forecast_results': None,
            'forecast_images': {},
            'forecast_performance_image': None
        }
        out = {'report': report_obj}
        out.update(memory_update(step_name, "Final report generated: insufficient data", errors=errors))
        return out

    # Normal success path - include all visualizations and results
    try:
        rf_metrics = {}; dt_metrics = {}
        if forecast_results and 'methods' in forecast_results:
            rf_metrics = forecast_results['methods'].get('random_forest', {}).get('metrics', {})
            dt_metrics = forecast_results['methods'].get('decision_tree', {}).get('metrics', {})
        fe_insights = "" if not feature_engineering_applied else f"\n- Feature Engineering: Applied | Reason: {feature_engineering_reason} | Original features: {original_feature_count}, Final features: {feature_count}"
        if not feature_engineering_applied:
            fe_insights = f"\n- Feature Engineering: Skipped | Reason: {feature_engineering_reason}"
        figure_descriptions = """VISUALIZATIONS GENERATED:\n- Figure 1 (Power vs Time)\n- Figure 2 (Correlation Matrix)\n- Figure 3 (Forecast Performance)"""
        suitable_prompt = f"""You are a data analyst reporting on time series analysis results. Summarize this dataset and forecasting analysis in 3 paragraphs:

**IMPORTANT DOMAIN CONTEXT:**
{data_context_str}

**DATA SUMMARY:**
- Total rows: {total_rows}, Timespan: {timespan_days:.1f} days
- Features: {feature_count} (after preprocessing)
- {missing_info}{fe_insights}

**FORECASTING RESULTS:**
- Recommended approach: {recommended_method}
- Random Forest: MAE={rf_metrics.get('MAE',0):.2f}, RMSE={rf_metrics.get('RMSE',0):.2f}, R²={rf_metrics.get('R2',0):.3f}
- Decision Tree: MAE={dt_metrics.get('MAE',0):.2f}, RMSE={dt_metrics.get('RMSE',0):.2f}, R²={dt_metrics.get('R2',0):.3f}

{figure_descriptions}

REQUIREMENTS:
1. First paragraph: describe dataset characteristics, data quality (missing values, preprocessing applied), feature engineering decision and reason
2. Second paragraph: compare the two models' performance (MAE, RMSE, R², which is better and why), briefly describe visualizations produced
3. Third paragraph: recommend next steps (validation approach, model selection, limitations to address)

Keep it technical and concise (3-4 sentences per paragraph). DO NOT fabricate information."""
        llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
        response = llm.invoke([HumanMessage(content=suitable_prompt)])
        summary_text = response.content
    except Exception as e:
        summary_text = f"Analysis complete. Dataset: {total_rows} rows, {timespan_days:.1f} days. Models evaluated."

    # Extract the main forecast performance image from forecast_images dict
    forecast_performance_image = forecast_images.get('test_performance') if forecast_images else None
    
    report_obj = {
        'status': 'success',
        'forecast_summary': summary_text,
        'forecast_results': forecast_results,
        'forecast_images': forecast_images,
        'forecast_performance_image': forecast_performance_image,
        'ts_analysis': ts_analysis,
        'ts_method': state.get('ts_method'),
        'ts_suitable': state.get('ts_suitable'),
        'data_profile': data_profile,
        'summary_stats': summary_stats,
        'engineered_data': engineered_data,
        'visualizations': visualizations,
        'viz_images': viz_images,
        'data_context': data_context  # CRITICAL: Preserve data_context in report for visibility
    }
    out = {'report': report_obj}
    out.update(memory_update(step_name, "Final report generated: success"))
    return out

## Workflow with Reflection Loops

This workflow integrates reflection nodes for iterative improvement through observe-reflect-plan-act cycles.

In [ ]:
# REFLECTIVE WORKFLOW CONFIGURATION (v2)
from langgraph.graph import StateGraph

# Conditional routing functions (ensure they're in scope)
def check_after_load(state):
    if state.get("stop_processing", False) or state.get("data") is None:
        return "final_report"
    return "validate"

def check_after_validate(state):
    if state.get("stop_processing", False):
        return "final_report"
    return "analyze"

def check_after_llm_validate(state):
    """Critical gate: if data is insufficient, skip ALL processing and go directly to final_report"""
    if state.get("stop_processing", False):
        return "final_report"
    return "preprocess"

def check_after_data_reflection(state):
    """Check if data quality needs revision. Hard limit at 1 revision."""
    # EARLY EXIT: Skip if stop_processing is set
    if state.get("stop_processing", False):
        return "final_report"
    
    needs_revision = state.get("needs_revision", False)
    revision_count = state.get("revision_count", 0)
    max_data_revisions = 1
    
    if revision_count >= max_data_revisions:
        return "feature_eng"  # Hard exit after max revisions
    
    if needs_revision:
        return "preprocess"  # Retry
    
    return "feature_eng"  # Good quality, move on

def check_after_analysis_reflection(state):
    """Check if analysis quality needs revision. Hard limit at 1 revision."""
    # EARLY EXIT: Skip if stop_processing is set
    if state.get("stop_processing", False):
        return "final_report"
    
    needs_revision = state.get("needs_revision", False)
    revision_count = state.get("revision_count", 0)
    max_analysis_revisions = 1
    
    if revision_count >= max_analysis_revisions:
        return "visualize"  # Hard exit after max revisions
    
    if needs_revision:
        return "feature_eng"  # Retry
    
    return "visualize"  # Good quality, move on

def check_after_forecast_reflection(state):
    """
    Check if forecast quality needs revision. Hard limit at 2 revisions.
    This is the critical function that prevents infinite loops.
    """
    # EARLY EXIT: Skip if stop_processing is set
    if state.get("stop_processing", False):
        return "final_report"
    
    needs_revision = state.get("needs_revision", False)
    revision_count = state.get("revision_count", 0)
    quality_score = state.get("forecast_quality_score", 0.5)
    max_revisions = 2
    
    print(f"\n[check_after_forecast_reflection] Decision point:")
    print(f"  revision_count={revision_count} (max={max_revisions})")
    print(f"  quality_score={quality_score:.3f}")
    print(f"  needs_revision={needs_revision}")
    
    # CRITICAL: Hard stop after max revisions to prevent infinite loops
    if revision_count >= max_revisions:
        print(f"  → Max revisions reached. Exiting to final_report.")
        return "final_report"
    
    # Only retry if quality is still poor
    if needs_revision and quality_score < 0.4:
        print(f"  → Quality too low. Looping back to ts_train for revision {revision_count + 1}/{max_revisions}")
        return "ts_train"
    
    print(f"  → Quality acceptable or no revision needed. Exiting to export_images.")
    return "export_images"

# Build new workflow instance (will reuse existing EDAState schema from memory)
reflective_workflow = StateGraph(EDAState)

# Add processing nodes
reflective_workflow.add_node("load", load_data)
reflective_workflow.add_node("validate", validate_data)
reflective_workflow.add_node("analyze", analyze_data)
reflective_workflow.add_node("llm_validate", llm_validate_data_sufficiency)
reflective_workflow.add_node("preprocess", data_preprocessing)
reflective_workflow.add_node("feature_eng", feature_engineering)
reflective_workflow.add_node("visualize", visualize_data)
reflective_workflow.add_node("ts_suitable", analyze_timeseries_suitability)
reflective_workflow.add_node("ts_train", train_forecast_model)
reflective_workflow.add_node("ts_visualize", visualize_forecast_results)
reflective_workflow.add_node("final_report", generate_final_report)

# Add checkpoint marker nodes
reflective_workflow.add_node("cp_preprocess", lambda s: mark_checkpoint(s, "preprocess"))
reflective_workflow.add_node("cp_feature", lambda s: mark_checkpoint(s, "feature_eng"))
reflective_workflow.add_node("cp_forecast", lambda s: mark_checkpoint(s, "forecast"))

# Add reflection nodes
reflective_workflow.add_node("reflect_data", reflect_on_data_quality)
reflective_workflow.add_node("reflect_analysis", reflect_on_analysis_quality)
reflective_workflow.add_node("reflect_forecast", reflect_on_forecast_quality)

# Add guardrail validation node
reflective_workflow.add_node("guardrail", guardrail_validation)

# Add export images node (vision analysis moved to main execution cell for cleaner output)
def export_images_node(state: EDAState) -> dict:
    return export_images_to_workspace(state)

reflective_workflow.add_node("export_images", export_images_node)

# Build edges: Phase 1 (load/validate/analyze/llm_gate)
# CRITICAL: llm_validate is the gateway - if insufficient data, jump directly to final_report
reflective_workflow.add_conditional_edges("load", check_after_load, {"validate": "validate", "final_report": "final_report"})
reflective_workflow.add_conditional_edges("validate", check_after_validate, {"analyze": "analyze", "final_report": "final_report"})
reflective_workflow.add_edge("analyze", "llm_validate")
reflective_workflow.add_conditional_edges("llm_validate", check_after_llm_validate, {"preprocess": "preprocess", "final_report": "final_report"})

# Phase 2: Preprocessing + checkpoint + reflection loop
# If stop_processing is set, reflection will route to final_report
reflective_workflow.add_edge("preprocess", "cp_preprocess")
reflective_workflow.add_edge("cp_preprocess", "reflect_data")
reflective_workflow.add_conditional_edges("reflect_data", check_after_data_reflection, {"preprocess": "preprocess", "feature_eng": "feature_eng", "final_report": "final_report"})

# Phase 3: Feature engineering + checkpoint + reflection loop
reflective_workflow.add_edge("feature_eng", "cp_feature")
reflective_workflow.add_edge("cp_feature", "reflect_analysis")
reflective_workflow.add_conditional_edges("reflect_analysis", check_after_analysis_reflection, {"feature_eng": "feature_eng", "visualize": "visualize", "final_report": "final_report"})

# Phase 4: Visualization + training + checkpoint + forecast reflection + guardrail
reflective_workflow.add_edge("visualize", "ts_suitable")
reflective_workflow.add_edge("ts_suitable", "ts_train")
reflective_workflow.add_edge("ts_train", "ts_visualize")
reflective_workflow.add_edge("ts_visualize", "cp_forecast")
reflective_workflow.add_edge("cp_forecast", "reflect_forecast")
reflective_workflow.add_edge("reflect_forecast", "guardrail")
reflective_workflow.add_conditional_edges("guardrail", check_after_forecast_reflection, {"ts_train": "ts_train", "export_images": "export_images", "final_report": "final_report"})

# Export images and proceed to final report (vision analysis done in main execution cell)
reflective_workflow.add_edge("export_images", "final_report")

# Entry/finish
reflective_workflow.set_entry_point("load")
reflective_workflow.set_finish_point("final_report")

# Compile with checkpointer
reflective_agent = reflective_workflow.compile(checkpointer=checkpointer)

In [ ]:
from IPython.display import Image, display

# Visualize the LangGraph structure
display(Image(reflective_agent.get_graph(xray=True).draw_mermaid_png()))

# Run Extended Workflow

In [ ]:
import time, uuid, os, pandas as pd, base64, textwrap, io, contextlib
from IPython.display import Image, display, Markdown

wind_filepath = "/Users/albert/Desktop/lab7/data/Solar station site 1 (Nominal capacity-50MW)(1).xlsx"

# Define data context for domain-aware analysis
data_context = {
    "description": "solar power output data",
    "resolution": "30 minutes",
    "goal": "Day-ahead forecasting",
    "capacity": "50 MW"
}

# Helper to run a function quietly (suppress internal prints)
def run_quiet(func, *args, **kwargs):
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        result, sid = func(*args, **kwargs)
    return result, sid, buf.getvalue()

# ---- Reflective workflow run (quiet) ----
refl_session_id = str(uuid.uuid4())
start_refl = time.time()
result_reflective, sid_reflective, refl_log = run_quiet(run_reflective_analysis, wind_filepath, refl_session_id, data_context)
refl_elapsed = time.time() - start_refl
thread_refl = {"configurable": {"thread_id": refl_session_id}}
refl_cps = list(checkpointer.list(thread_refl))

# Extract reports
report_refl = (result_reflective.get("report") or {})
forecast_refl = report_refl.get("forecast_results") or {}
methods_refl = forecast_refl.get("methods", {}) if forecast_refl else {}
rf_refl = methods_refl.get("random_forest", {}).get("metrics", {}) if methods_refl else {}

# Reflection field check
reflection_fields = ["reflections","critique_history","revision_count","data_quality_score","analysis_quality_score","forecast_quality_score"]
reflection_presence = {f: (f in result_reflective) for f in reflection_fields}
reflection_ok = all(reflection_presence.values())

# Status + checkpoints
refl_status = report_refl.get("status","unknown")
allowed_status = {"success","completed","insufficient_data"}
refl_cp_ok = len(refl_cps) > 0
serialization_ok = True
try:
    test_df = pd.DataFrame({"x":[1,2,3]})
    ser_tuple = checkpointer.serde.dumps_typed(test_df)
    df_round = checkpointer.serde.loads_typed(ser_tuple)
    serialization_ok = df_round.equals(test_df)
except Exception:
    serialization_ok = False

all_ok = (refl_status in allowed_status and refl_cp_ok and serialization_ok)

# CRITICAL: Check if workflow stopped due to insufficient data
stop_processing = result_reflective.get('stop_processing', False)

# ---- Display ALL visualization images (ONLY for successful runs) ----
if refl_status == "success":
    # 1. Display EDA visualizations (Power vs Time, Correlation Matrix, etc.)
    viz_images_refl = report_refl.get('viz_images', {})
    if viz_images_refl:
        for fig_key, img_data in viz_images_refl.items():
            try:
                display(Image(data=base64.b64decode(img_data)))
            except Exception as e:
                pass

    # 2. Display forecast-related images (if any from forecast_images dict)
    forecast_images_refl = report_refl.get('forecast_images', {})
    if forecast_images_refl:
        for fig_key, img_data in forecast_images_refl.items():
            try:
                display(Image(data=base64.b64decode(img_data)))
            except Exception as e:
                pass

    # 3. Display main forecast performance image
    forecast_perf_img = report_refl.get('forecast_performance_image')
    if forecast_perf_img:
        try:
            display(Image(data=base64.b64decode(forecast_perf_img)))
        except Exception as e:
            pass

# ---- Report text outputs (reflective) ----
refl_report_text = report_refl.get('forecast_summary')

def format_report(title, text):
    if not text: return f"### {title}\n(No content)"
    wrapped = textwrap.fill(text, width=110)
    return f"### {title}\n{wrapped}"

if refl_report_text:
    qs_line = ""
    # Only show quality scores if status is success (not insufficient_data)
    if 'data_quality_score' in result_reflective and refl_status == "success":
        qs_line = f"\n**Quality Scores**: Data={result_reflective.get('data_quality_score',0):.2f} Analysis={result_reflective.get('analysis_quality_score',0):.2f} Forecast={result_reflective.get('forecast_quality_score',0):.2f}"
    display(Markdown(format_report("Reflective Workflow Report" + qs_line, refl_report_text)))

# Save report texts
try:
    if refl_report_text:
        with open("reflective_report.txt","w",encoding="utf-8") as f: f.write(refl_report_text)
except Exception:
    pass

# Guardrail validation results
guardrail_passed = report_refl.get('guardrail_passed', True)
guardrail_errors = report_refl.get('guardrail_errors', [])

# Vision analysis for ALL generated images (ONLY if status is success)
if refl_status == "success":
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import HumanMessage
    vision_llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    try:
        # Analyze EDA visualizations (Power vs Time, Correlation Matrix)
        viz_images = report_refl.get('viz_images', {})
        if viz_images:
            for fig_key, img_data in viz_images.items():
                prompt = f"Describe this visualization in 2-3 sentences focusing on key patterns, trends, or insights for renewable energy forecasting. Be concise and technical."
                message = HumanMessage(
                    content=[
                        {"type": "text", "text": prompt},
                        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_data}"}}
                    ]
                )
                response = vision_llm.invoke([message])
                print(response.content)
        
        # Analyze forecast performance image (Figure 3)
        forecast_perf_img = report_refl.get('forecast_performance_image')
        if forecast_perf_img:
            prompt = f"Describe this forecast performance visualization in 2-3 sentences. Focus on: 1) How well the models fit the training and test data, 2) Comparison of Random Forest vs Decision Tree performance, 3) Any visible overfitting or underfitting patterns. Be concise and technical."
            message = HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{forecast_perf_img}"}}
                ]
            )
            response = vision_llm.invoke([message])
            print(response.content)
            
    except Exception:
        import traceback
        traceback.print_exc()